In [ ]:
import urllib.request
import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from collections import OrderedDict

In [ ]:
def read_csv(time_clock):
    sum_df = pd.read_csv("new_trend_filter/"+time_clock+"trend.csv", encoding='ms949')
    return sum_df


def filter_df(sum_df):
    enter_list = ['쇼미', '우승자', '문화의날','승국이','문화의 날','차민근','정국', '타투이스트', '날 녹여주오','언니','엄마','아빠','공항','우혜미','디원스','문보미','델루나','광대들','풍문조작단','남동생','여동생','동생','형','누나','친구','송자호', '장사리','작가','박일','염따','식객','설리','구혜선','안재현','구혜선','특선영화','쇼미더머니','쿠키','강다니엘','지효','보람튜브','철구','외질혜','감스트','밴쯔','이터널스','sbs', 'kbs', 'mbc', 'jtbc','tvn','멜로가 체질', '미스터 기간제', '의사요한', '시즌','솜해인', '호텔 델루나', '아이돌학교', '공항패션','프로듀스', '워너원', '강타', '정유미', '그것이 알고싶다', '가수', '동물농장','하트시그널', '꽃보다', '강식당', '삼시세끼', 'tvn', '생활의달인', 'vj특공대', '유튜버', 'bj', '복면가왕', '골목식당', '서민갑부', 'KBS', 'SBS', "MBC", "결혼", "연애", "열애", '1박2일','런닝맨','라디오스타','개그맨','배우','뮤지컬','개봉','시사회']
    poli_list = ['미투','백색국가','가짜뉴스','홍정욱','라치몬트','방사포','탄도미사일','나경원','법','촛불','집회','촛불집회','광화문','서초동','윤지오','이용마','비무장지대','잠만경','일본', '변호사', '북한', '미사일', '문재인', '황교안', '이낙연', '트럼프', '조국', '청문회', '공청회', '탄핵', '박근혜', '구형', '파업']
    crime_list = ['버닝썬', '검찰','고유정','신유용','조은누리','사망','티팬티남','로빈장', '더페스타', '폭행', '사형', '무기징역', '실종', '살해', '살인', '강간', '구형','자살', '사건', '사고', '방화', '화재', '폭발', '사망']
    study_list = ['수능',' 모의고사', '시험', '접수']
    etc_list = ['류민근', '역바이럴','광주클럽', '클럽', '충북 여교사', '박경락', '평양냉면','정기휴일','휴무','평양냉면','서버시간', '티켓팅', '뜻', '나이', '남편', '아내', '이유', '최저임금', '2019', '2020', '문화센터']
    sport_list = ['황바울','양준혁','한국','시구','정병국','류현진', '메이저리그', '중계', '축구', '손흥민', '국가대표', '올림픽','메달', '호날두', '프리미어리그', '야구', '유벤투스', '뮌헨', '토트넘', '리버풀', '챔스']
    nature_list = ['미탁','붕괴','링링','위치','북상','침수','입추','동지','하지','경칩','초복','중복','말복','태풍', '지진', '날씨', '우박']
    economic_list = ['화재','남대문오피스텔', '퇴계로27', 'dhc제품', '노노재팬','증시', '주가', '금리', 'sk', '삼성', '애플', 'lg', '현대', '아이폰', '갤럭시', '로또']
    except_list = enter_list + poli_list + crime_list + study_list + etc_list + sport_list + nature_list + economic_list
    
    ad_list1, ad_list2 = dict(), dict()
    for i in range(3, len(sum_df.columns)):

        keyword = sum_df.columns[i]
        temp = False
        for except_word in except_list:
            if except_word in keyword:
                temp = True
                break
            else:
                pass

        if temp:
            continue

        trend_keyword = sum_df[keyword]
        numberZero = 0
        for k in range(len(sum_df.index)-2):
            # 0인 갯수 수집
            if trend_keyword[k] != 0:
                numberZero+=1

            # 0이었다가 다음날 100이면 광고
            if (trend_keyword[k] <= 0.01) and (trend_keyword[k+1] >= 99):
                rank_date = sum_df['기간'][k]
                rank_date = rank_date.replace("-","")
                rank_date_year = rank_date[:4]
                if rank_date_year == '2019':
                    rank_date = int(rank_date)
                    if rank_date in ad_list1:
                        ad_list1[rank_date].append(keyword)
                    else:
                        ad_list1[rank_date] = [keyword]
                    break



        # 검색데이터가 sum_df.index중에서 200개 미만 있는 경우
        # ad_list1에 없는 경우에 한하여 추가
        if numberZero < 300:
            if keyword not in ad_list1:
                # 광고날짜 찾기
                trend_keyword = sum_df[keyword]
                for j in range(len(sum_df.index)-2):
                    if trend_keyword[j] >= 99:
                        rank_date = sum_df['기간'][j]
                        rank_date = rank_date.replace("-","")
                        rank_date_year = rank_date[:4]
                        if rank_date_year == '2019':
                            rank_date = int(rank_date)
                            if rank_date in ad_list2:
                                ad_list2[rank_date].append(keyword)
                            else:
                                ad_list2[rank_date] = [keyword]
                            break
    
    ad_list = dict()
    for date, val_list in ad_list1.items():
        for date2, val_list2 in ad_list2.items():
            if date == date2:
                ad_list[date] = val_list + val_list2
            else:
                ad_list[date] = val_list
                ad_list[date2] = val_list2
    ad_list = sorted(ad_list.items())
    return ad_list

def framing(result_list):
    df =pd.DataFrame(result_list, columns=['날짜','급상승 검색어'])
    return df

def counting_keywordNum(dict_file):
    len_ad_dict = {}
    for i in dict_file:
        len_ad_dict[i[0]] = len(i[1])
    len_ad_dict = sorted(len_ad_dict.items())
    date_df =pd.DataFrame(len_ad_dict, columns=['날짜', '실검 갯수'])
    return date_df

def joiningDf(df1, df2):
    keyword_list_num_df = df1.join(df2.set_index("날짜"), on = "날짜")
    return keyword_list_num_df

def savingDf(df):
    df.to_csv("keyword_list_num_df_data/"+time_clock+"keyword_list_num_df.csv", encoding='ms949')
    
def count_adkeyword(df):
    count = 0
    for i in df['실검 갯수']:
        count += i
    return count

def saveTimeCount(tiemCount_dict, time_clock, count):
    tiemCount_dict[time_clock] = count
    return tiemCount_dict
